In [1]:
import os

In [2]:
#os.environ["JAVA_HOME"] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_152.jdk/Contents/Home/'
os.environ["PYSPARK_SUBMIT_ARGS"] = 'pyspark-shell'

In [3]:
os.environ["SPARK_HOME"] = '/home/dchernovol/Downloads/spark-2.2.0-bin-hadoop2.7'

In [4]:
import sys

sys.path.append(os.environ['SPARK_HOME']+"/python")

In [5]:
sys.path.append(os.environ['SPARK_HOME']+"/python/lib/py4j-0.10.4-src.zip")

In [6]:
import py4j
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext

In [7]:
conf = (SparkConf().setMaster("local[4]")
        .setAppName("ML demo")
        .set("spark.executor.memory", "1g"))

In [8]:
sc = SparkContext(conf=conf)

In [9]:
sqlcontext = SQLContext(sc)

In [10]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionModel, LinearRegressionWithSGD
import numpy as np

In [11]:
from pyspark.ml.classification import GBTClassificationModel

In [12]:
data=[
    LabeledPoint(0.0,[0.0]),
    LabeledPoint(1.0,[1.0]),
    LabeledPoint(3.0,[2.0]),
    LabeledPoint(2.0,[3.0])
]
lrm=LinearRegressionWithSGD.train(sc.parallelize(data),iterations=10,initialWeights=np.array([1.0]))
print(lrm.predict(np.array([1.0])))

/home/dchernovol/Downloads/spark-2.2.0-bin-hadoop2.7/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


0.928638123469


In [13]:
df = sqlcontext.read.format(
    'com.databricks.spark.csv').options(
    header='true').load('train.csv')

In [14]:
df.head(3)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A/5 21171', Fare=u'7.25', Cabin=None, Embarked=u'S'),
 Row(PassengerId=u'2', Survived=u'1', Pclass=u'1', Name=u'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex=u'female', Age=u'38', SibSp=u'1', Parch=u'0', Ticket=u'PC 17599', Fare=u'71.2833', Cabin=u'C85', Embarked=u'C'),
 Row(PassengerId=u'3', Survived=u'1', Pclass=u'3', Name=u'Heikkinen, Miss. Laina', Sex=u'female', Age=u'26', SibSp=u'0', Parch=u'0', Ticket=u'STON/O2. 3101282', Fare=u'7.925', Cabin=None, Embarked=u'S')]

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql import types

def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf = udf(Embarked_transform, types.StringType())
df = df.withColumn('Embarked', my_udf(df['Embarked']))
df.select('Embarked').distinct().collect()

[Row(Embarked=u'Q'), Row(Embarked=u'C'), Row(Embarked=u'S'), Row(Embarked=u'')]

In [16]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df_t = encoder.transform(indexed)

In [17]:
test_df = sqlcontext.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(test_df).transform(test_df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



In [18]:
def parse_age(str_age):
    try:
        return float(str_age)
    except:
        return -1

In [19]:
def transf(r):
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),
        ] + list(r.EmbarkedVec.toArray())
    )

In [20]:
data = df_t.rdd.map(transf)

In [21]:
data.take(1)

[LabeledPoint(0.0, [3.0,1.0,7.25,1.0,0.0,22.0,1.0,0.0,0.0])]

In [22]:
train, test = data.randomSplit([0.7, 0.3])

In [23]:
train.cache()

PythonRDD[84] at RDD at PythonRDD.scala:48

In [24]:
train.take(1)

[LabeledPoint(0.0, [3.0,1.0,7.25,1.0,0.0,22.0,1.0,0.0,0.0])]

In [25]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
rfc = RandomForest.trainClassifier(train, numClasses=2,
                             categoricalFeaturesInfo={},
                            numTrees=100)

In [26]:
def acc(m, test):
    values = test.map(lambda x: x.features)
    yhat = rfc.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    errors = comp.map(lambda x: abs(x[0]-x[1]))
    return 1-errors.sum()/errors.count()

In [27]:
acc(rfc, test)

0.8282442748091603

## Home task

In [28]:
# добавить 5 новых фичей
# 3 фичи высчитываются из имеющихся
# хотя бы одна использует udf

# попробовать 3 новых модели

# f1 меру

In [29]:
test.cache()

PythonRDD[118] at RDD at PythonRDD.scala:48

In [30]:
def precision(m, test):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    true_positive = comp.map(lambda x: 1 if (x[0] == 1 == x[1]) else 0)
    predicted_condition_positive = comp.map(lambda x: 1 if (x[0] == 1) else 0)
    return float(true_positive.sum())/predicted_condition_positive.sum()

In [31]:
precision(rfc, test)

0.9117647058823529

In [32]:
def recall(m, test):
    values = test.map(lambda x: x.features)
    yhat = m.predict(values)
    y = test.map(lambda x: x.label)
    comp = yhat.zip(y)
    true_positive = comp.map(lambda x: 1 if (x[0] == 1 == x[1]) else 0)
    condition_condition_positive = comp.map(lambda x: 1 if (x[1] == 1) else 0)
    return float(true_positive.sum())/condition_condition_positive.sum()

In [33]:
recall(rfc, test)

0.6138613861386139

In [34]:
def f1_score(m, test):
    return 2/(1/recall(m, test) + 1/precision(m, test))

In [35]:
f1_score(rfc,test)

0.7337278106508877

In [36]:
def parse_age(str_age):
    try:
        return float(str_age)
    except:
        return 0

In [37]:
def parse_cabin(str_cabin):
    return 0 if str_cabin == None else 1

In [38]:
print (parse_cabin(None))
print (parse_cabin('123'))

0
1


In [39]:
df = sqlcontext.read.format(
    'com.databricks.spark.csv').options(
    header='true').load('train.csv')
def Embarked_transform(x):
    if x != None:
        return x
    else:
        return ''

my_udf = udf(Embarked_transform, types.StringType())
df = df.withColumn('Embarked', my_udf(df['Embarked']))
df.select('Embarked').distinct().collect()

[Row(Embarked=u'Q'), Row(Embarked=u'C'), Row(Embarked=u'S'), Row(Embarked=u'')]

In [40]:
def ticket_transform(x):
    if '/' in x:
        x = x.split('/')[0]
        return x
    return 'none'

my_udf_ticket = udf(ticket_transform, types.StringType())
df = df.withColumn('Ticket', my_udf_ticket(df['Ticket']))
df.select('Ticket').distinct().collect()

[Row(Ticket=u'SC'),
 Row(Ticket=u'none'),
 Row(Ticket=u'SW'),
 Row(Ticket=u'WE'),
 Row(Ticket=u'SO'),
 Row(Ticket=u'SOTON'),
 Row(Ticket=u'W.'),
 Row(Ticket=u'S.C.'),
 Row(Ticket=u'SCO'),
 Row(Ticket=u'A.'),
 Row(Ticket=u'C.A.'),
 Row(Ticket=u'A'),
 Row(Ticket=u'S.O.'),
 Row(Ticket=u'W'),
 Row(Ticket=u'STON'),
 Row(Ticket=u'P'),
 Row(Ticket=u'S.W.')]

In [41]:
df

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [42]:
df_t

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string, EmbarkedIndex: double, EmbarkedVec: vector]

In [43]:
stringIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")
df_t = encoder.transform(indexed)


my_stringIndexer = StringIndexer(inputCol="Ticket", outputCol="TicketIndex")
my_model = my_stringIndexer.fit(df_t)
my_indexed = my_model.transform(df_t)
my_encoder = OneHotEncoder(inputCol="TicketIndex", outputCol="TicketVec")
my_df_t = my_encoder.transform(my_indexed)

In [44]:
def my_transf(r):
    return LabeledPoint(
        int(r.Survived),
        [
            int(r.Pclass),
            r.Sex == 'male',
            float(r.Fare),
            int(r.SibSp),
            int(r.Parch),
            parse_age(r.Age),
            int(len(r.Name)),  # 1
            parse_cabin(r.Cabin) # 2
        ] + list(r.EmbarkedVec.toArray()) \
          + list(r.TicketVec)  # 3 UDF
    )

In [45]:
from pyspark.mllib.classification import NaiveBayes, SVMWithSGD

In [46]:
df

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [47]:
df_t.take(1)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A', Fare=u'7.25', Cabin=None, Embarked=u'S', EmbarkedIndex=0.0, EmbarkedVec=SparseVector(3, {0: 1.0}))]

In [48]:
my_df_t.take(1)

[Row(PassengerId=u'1', Survived=u'0', Pclass=u'3', Name=u'Braund, Mr. Owen Harris', Sex=u'male', Age=u'22', SibSp=u'1', Parch=u'0', Ticket=u'A', Fare=u'7.25', Cabin=None, Embarked=u'S', EmbarkedIndex=0.0, EmbarkedVec=SparseVector(3, {0: 1.0}), TicketIndex=1.0, TicketVec=SparseVector(16, {1: 1.0}))]

In [49]:
my_data = my_df_t.rdd.map(my_transf)

In [50]:
my_data.take(1)

[LabeledPoint(0.0, [3.0,1.0,7.25,1.0,0.0,22.0,23.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])]

In [51]:
my_train, my_test = my_data.randomSplit([0.7, 0.3])

In [52]:
my_train.cache()
my_test.cache()

PythonRDD[230] at RDD at PythonRDD.scala:48

In [53]:
my_train.take(2)

[LabeledPoint(0.0, [3.0,1.0,7.25,1.0,0.0,22.0,23.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [3.0,1.0,8.05,0.0,0.0,35.0,24.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])]

In [54]:
my_NaiveBayesModel = NaiveBayes.train(my_train)

In [55]:
predictionAndLabel = my_train.map(lambda p: (my_NaiveBayesModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / my_train.count()
print('model accuracy {}'.format(accuracy))

model accuracy 0.656


In [56]:
from pyspark.mllib.classification import SVMWithSGD

In [57]:
my_SVMWithSGD = SVMWithSGD.train(my_train)

In [58]:
predictionAndLabel = my_train.map(lambda p: (my_SVMWithSGD.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / my_train.count()
print('model accuracy {}'.format(accuracy))

model accuracy 0.5632
